In [ ]:
!pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.

In [ ]:
# Necessary imports
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
from google.colab import drive
import pandas as pd
import os

In [ ]:
# Initialize MTCNN for face detection
mtcnn = MTCNN()

# Load pre-trained Inception ResNet model
resnet = InceptionResnetV1(pretrained='casia-webface').eval()

  0%|          | 0.00/111M [00:00<?, ?B/s]

In [ ]:
def run_model(img_path1, img_path2):

  # Load two face images to be verified
  img1 = Image.open(img_path1).convert('RGB') # Convert to RGB
  img2 = Image.open(img_path2).convert('RGB') # Convert to RGB

  # Detect faces and extract embeddings
  faces1, _ = mtcnn.detect(img1)
  faces2, _ = mtcnn.detect(img2)

  if faces1 is not None and faces2 is not None:
      aligned1 = mtcnn(img1)
      aligned2 = mtcnn(img2)

      aligned1 = aligned1.unsqueeze(0)  # Adds a dimension of size 1 at the beginning
      aligned2 = aligned2.unsqueeze(0)

      embeddings1 = resnet(aligned1).detach()
      embeddings2 = resnet(aligned2).detach()

      # Calculate the Euclidean distance between embeddings
      distance = (embeddings1 - embeddings2).norm().item()
      if distance < 1.0:  # Adjustable threshold for verification
          # Same Person
          return 1
      else:
          # Different Persons
          return 0
  else:
      return -1

In [ ]:
drive.mount('/content/drive')

csv_path = '/content/drive/MyDrive/processed_images/classification_summary.csv'
df = pd.read_csv(csv_path)


# Separate data by image label into separate dataframes
sober_images = df[df['Label'] == 'sober']
one_glass_images = df[df['Label'] == 'one_glass']
three_glasses_images = df[df['Label'] == 'three_glasses']

# Function to determine if the model was correct
def validate_result(person_1, person_2, result):
    if person_1 == person_2 and result == 1:
        return 1  # Correctly identified as the same person
    elif person_1 != person_2 and result == 0:
        return 1  # Correctly identified as different people
    else:
        return 0  # Incorrect result

# Function to compare images within a label group
def test_batch(group1, group2):
    results = []
    for _, row1 in group1.iterrows():
        img_path1 = row1['Image']
        person_1 = row1['Person']

        group2_dict = group2.groupby('Person')['Image'].apply(list).to_dict()
        same_person_list = group2_dict[person_1]
        for same_person_image in same_person_list:
          result = run_model(img_path1, same_person_image)
          results.append(validate_result(person_1, person_1, result))

        random_samples = group2.sample(n=min(10, len(group2)), random_state=42)
        for _, row2 in random_samples.iterrows():
            img_path2 = row2['Image']
            person_2 = row2['Person']
            if person_1 == person_2:
              continue
            else:
              result = run_model(img_path1, img_path2)
              results.append(validate_result(person_1, person_2, result))

    return results


# This tests the accuracy of the model with a control group comparing sober images and one glass
results_control = []
results_control.extend(test_batch(sober_images, one_glass_images))
results_control.extend(test_batch(sober_images, sober_images))
control_accuracy = sum(results_control) / len(results_control)
print(f"Control Accuracy: {control_accuracy}")

# This tests the accuracy of the model comparing sober images to drunk, and drunk amongst themselves
results_drunk = []
results_drunk.extend(test_batch(sober_images, three_glasses_images))
results_drunk.extend(test_batch(three_glasses_images, three_glasses_images))
drunk_accuracy = sum(results_drunk) / len(results_drunk)
print(f"Drunk Accuracy: {drunk_accuracy}")

Mounted at /content/drive
Control Accuracy: 0.9982547993019197
Drunk Accuracy: 0.9956369982547993


https://medium.com/@danushidk507/facenet-pytorch-pretrained-pytorch-face-detection-mtcnn-and-facial-recognition-b20af8771144